<a href="https://colab.research.google.com/github/hurtadosanti/TrafficSignClassifier/blob/master/Tfv2.TrafficSignClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Traffic Sign Recognition Classifier
Deep Learning

**TensorFlow v2**

[Problem Definition](https://github.com/udacity/CarND-Traffic-Sign-Classifier-Project/blob/master/Traffic_Sign_Classifier.ipynb)

https://www.tensorflow.org/tutorials/keras/classification#evaluate_accuracy

## Import packages

In [10]:
import numpy as np
import pandas as pd
import pickle
import os
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
print(tf.__version__)
%matplotlib inline


2.3.0


## Load Data

In [11]:
traffic_signs_file = 'traffic-signs-data.zip'
!wget --no-check-certificate --progress=bar:force:noscroll \
    https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip \
    -O traffic-signs-data.zip

!wget --no-check-certificate --progress=bar:force:noscroll \
    https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv -O signnames.csv


--2020-10-02 07:01:21--  https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.120.224
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.120.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123524425 (118M) [application/zip]
Saving to: ‘traffic-signs-data.zip’

traffic-signs-data. 100%[===================>] 117.80M  19.6MB/s    in 7.1s    

2020-10-02 07:01:29 (16.5 MB/s) - ‘traffic-signs-data.zip’ saved [123524425/123524425]

--2020-10-02 07:01:29--  https://raw.githubusercontent.com/udacity/CarND-Traffic-Sign-Classifier-Project/master/signnames.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 999 [text

In [12]:
with ZipFile(traffic_signs_file, 'r') as zipObj:
   zipObj.extractall()
   print('Finish extracting')

Finish extracting


In [13]:
train = pickle.load(open("train.p", 'rb'))
valid=pickle.load(open("valid.p", 'rb'))
test = pickle.load(open("test.p", 'rb'))
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']
# Verify that the data is complete
assert(len(X_train)==len(y_train))
assert(len(X_valid)==len(y_valid))
assert(len(X_test)==len(y_test))

assert(len(X_train)>0)
assert(len(X_valid)>0)
assert(len(X_test)>0)
print('Data is correct')

Data is correct


## Data preprocessing

In [14]:
X_train_normalized = X_train/255.0
X_test_normalized = X_test/255.0
X_valid_normalized = X_valid/255.0

X_train_gray = np.sum(X_train_normalized/3,axis=3,keepdims=True)
X_test_gray = np.sum(X_test_normalized/3,axis=3,keepdims=True)
X_valid_gray = np.sum(X_valid_normalized/3,axis=3,keepdims=True)

image_shape = X_train_gray[0].shape
print(image_shape)

(32, 32, 1)


## Training
### Network Architecture

### Prepare the model

In [15]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_shape))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(43,activation='sigmoid'))


### Execute the training
We stop when accuracy is over 0.95

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 30, 30, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2048)             

In [17]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(X_train_gray, y_train,
                    epochs=20,batch_size=128,
                    shuffle=True,
                    validation_data=(X_valid_gray, y_valid))


Epoch 1/20
272/272 [==============================] - 2s 6ms/step - loss: 3.4567 - accuracy: 0.0734 - val_loss: 3.2370 - val_accuracy: 0.0782
Epoch 2/20
272/272 [==============================] - 1s 5ms/step - loss: 3.1240 - accuracy: 0.3191 - val_loss: 3.0205 - val_accuracy: 0.5542
Epoch 3/20
272/272 [==============================] - 1s 5ms/step - loss: 3.0031 - accuracy: 0.5621 - val_loss: 2.9537 - val_accuracy: 0.6726
Epoch 4/20
272/272 [==============================] - 1s 5ms/step - loss: 2.9566 - accuracy: 0.6461 - val_loss: 2.9283 - val_accuracy: 0.7200
Epoch 5/20
272/272 [==============================] - 1s 5ms/step - loss: 2.9302 - accuracy: 0.6816 - val_loss: 2.9142 - val_accuracy: 0.7283
Epoch 6/20
272/272 [==============================] - 1s 5ms/step - loss: 2.9138 - accuracy: 0.7044 - val_loss: 2.9036 - val_accuracy: 0.7338
Epoch 7/20
272/272 [==============================] - 1s 5ms/step - loss: 2.9021 - accuracy: 0.7317 - val_loss: 2.8997 - val_accuracy: 0.7585
Epoch 

## Test the model

In [19]:
test_loss, test_acc = model.evaluate(X_test_gray,  y_test)

print('\nTest accuracy:', test_acc)

395/395 [==============================] - 1s 2ms/step - loss: 2.8829 - accuracy: 0.8829

Test accuracy: 0.8828978538513184
